In [1]:
!pip install git+https://github.com/DeepLabCut/DeepLabCut.git

  Cloning https://github.com/DeepLabCut/DeepLabCut.git to /tmp/pip-req-build-sl3ppi41
  Running command git clone -q https://github.com/DeepLabCut/DeepLabCut.git /tmp/pip-req-build-sl3ppi41
     |████████████████████████████████| 177 kB 5.1 MB/s 
     |████████████████████████████████| 109 kB 50.9 MB/s 
     |████████████████████████████████| 948 kB 52.6 MB/s 
     |████████████████████████████████| 3.3 MB 39.8 MB/s 
     |████████████████████████████████| 11.2 MB 37.1 MB/s 
     |████████████████████████████████| 29.2 MB 84.7 MB/s 
     |████████████████████████████████| 9.8 MB 33.7 MB/s 
     |████████████████████████████████| 296 kB 43.5 MB/s 
     |████████████████████████████████| 352 kB 35.5 MB/s 
     |████████████████████████████████| 899 kB 44.6 MB/s 
     |████████████████████████████████| 34.5 MB 16 kB/s 
     |████████████████████████████████| 546 kB 49.2 MB/s 
     |████████████████████████████████| 462 kB 35.7 MB/s 
  Created wheel for deeplabcut: filename=deeplabcut-2.2.

In [2]:
import deeplabcut as dlc

from google.colab import drive
drive.mount('/content/drive/')

path = '/content/drive/MyDrive/DLC/'
path_config_file = path+'Bandit-Harris-2022-03-01/config.yaml'

# ---

import sys
sys.path.insert(0, '/content/drive/MyDrive/DLC')

import os
import shutil
import numpy as np
import pandas as pd
import touchscreen_toolbox as tb

FOLDER = path + '/tests'

# 
SAVE_STDOUT = sys.stdout  
def reset_std(): sys.stdout = SAVE_STDOUT

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
for (folder_path, _, files) in list(os.walk(FOLDER)):

    if files:
        
        # jump to next folder if no valid videos
        videos = [f for f in files if f.endswith('.mp4') and 
              (not tb.preprocess.is_preprocess(f))]
        if not videos:
            continue
        
        print(f"Processing folder {folder_path}")

        # create folder for dlc products
        dlc_folder = os.path.join(folder_path, 'DLC')
        tb.utils.mk_dir(dlc_folder)

        # analyze each video
        processed_ls = []
        for v in videos:
            v = os.path.join(folder_path, v)
            print(f"Processing {v}")

            # preprocess if needed
            name = tb.preprocess.preprocess(v)

            # call DLC to analyze video
            tb.analyze(path_config_file, name)

            # remove h5 & pickle files from dlc
            # csv will be renamed to the same as the video
            tb.cleanup(folder_path, dlc_folder, name)

            processed_ls.append(tb.utils.path2file(name))
            print("------------")

        # postprocess
        tb.postprocess.postprocess(folder_path)